In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from pandas_profiling import ProfileReport
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot , iplot,init_notebook_mode
import plotly.express as px
init_notebook_mode(connected = True)

In [4]:
df = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
df.info()

In [5]:
#The are some null values in data set
missing_count_percent = df.isnull().sum() * 100 / df.shape[0]
missing_count_percent  
df_missing_value = df.isnull().sum()
df_missing_value

In [6]:
df_missing_count_percent = pd.DataFrame(missing_count_percent)
df_missing_count_percent = df_missing_count_percent.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_Percentage (%)'
                    }
                )

In [7]:
df_missing_value = df.isnull().sum()
df_missing_value = df_missing_value.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_value_count'
                    }
                )

In [8]:
Final = df_missing_value.merge(df_missing_count_percent, how = 'inner', left_on = 'Column', right_on = 'Column')
Final

In [9]:
ProfileReport(df)

In [10]:
!pip install autoviz

Handling the missing data and deleting duplicates
It is important to handle missing data because any statistical results based on a dataset with non-random missing values could be biased. So you really want to see if these are random or non-random missing values.

Drop the columns which has high number of missing values.

We can impute(filling the missing values using the available information such as mean, median) but we should carefully see the pattern of the column before doing imputation.

For example - You want to fill the height of a person who male. Simpley adding 0 in the missing column would not make sense. So we can take the averega of male height and use that value inplace of missing values.

Rating - manually filling the data usin data from Netflix website

Country - replacing blank countries with the most common country

Cast - replacing null value with "Data not available"

Director - replacing null value with "Data not available"

In [11]:

df['rating'] = df['rating'].replace({'74 min': 'TV-MA', '84 min': 'TV-MA', '66 min': 'TV-MA'})
df['rating'] = df['rating'].replace({'TV-Y7-FV': 'TV-Y7'})

In [12]:
# Renaming vaules for Rating for better understanding
# Source : https://help.netflix.com/en/node/2064
df['rating'] = df['rating'].replace({
                'PG-13': 'Teens - Age above 12',
                'TV-MA': 'Adults',
                'PG': 'Kids - with parental guidence',
                'TV-14': 'Teens - Age above 14',
                'TV-PG': 'Kids - with parental guidence',
                'TV-Y': 'Kids',
                'TV-Y7': 'Kids - Age above 7',
                'R': 'Adults',
                'TV-G': 'Kids',
                 'G': 'Kids',
                'NC-17': 'Adults',
                'NR': 'NR',
                'UR' : 'UR'
                
})

In [13]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

df['cast'].replace(np.nan, 'No Data',inplace  = True)
df['director'].replace(np.nan, 'No Data',inplace  = True)
df.dropna(inplace=True)

# Drop Duplicates
df.drop_duplicates(inplace= True)

In [14]:
# splitting the genres in different rows to use it in the viz later

#df_genre = df[df['title'].isin(['Blood & Water', 'Dick Johnson Is Dead', 'Ganglands' ])]
df_genre = df[['show_id', 'title','type', 'listed_in' ]]
df_genre = (df_genre.drop('listed_in', axis=1)
             .join
             (
             df_genre.listed_in
             .str
             .split(', ',expand=True)
             .stack()
             .reset_index(drop=True, level=1)
             .rename('listed_in')           
             ))

> **************Creating new columns

In [16]:

df['month'] = pd.DatetimeIndex(df['date_added']).month

df_count = df['show_id'].count().sum()
print(df_count)
# Split of showes and TV
df_type = df.groupby('type')['show_id'].count().reset_index()
df_type = df_type.rename(columns = {"show_id":"count_showids"})

In [17]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Indicator(
    value = df_count))

fig = fig.update_layout(
        template = {'data' : {'indicator': [{
        'title': {'text': "Total content on Netflix"},}]
        }})

fig = fig.update_layout(
    #autosize=False,
    #width=500,
    height=100,
    margin=dict(l=50,r=50,b=0,t=1),)

# fig2 = px.pie(df_type, values='count_showids', names='type', color_discrete_sequence=px.colors.sequential.RdBu,
#        title='What type of titles are uploaded more on Netflix' , width=500, height=450)

fig.show()
#fig2.show()


fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]])
fig.add_trace(
    
    go.Bar(x= df_type['count_showids'], y= df_type['type'], orientation = 'h', marker=dict(color=["Maroon", "Grey"]), showlegend=False, 
           text = df_type['count_showids'], textposition='auto'),
    row=1, col=1)

fig.add_trace(
    
    go.Pie(labels=df_type['type'], values=df_type['count_showids'], marker_colors= ["Maroon", "Grey"]),
    row=1, col=2)

fig.update_layout(
    title_text="'What type of content is more uploaded more on Netflix?")
fig.show()

In [18]:
# splitting the countries in different rows 
#df_genre = df[df['title'].isin(['Blood & Water', 'Dick Johnson Is Dead', 'Ganglands' ])]
df_country = df[['show_id', 'title','type', 'country' ]]
df_country = (df_country.drop('country', axis=1)
             .join
             (
             df_country.country
             .str
             .split(', ',expand=True)
             .stack()
             .reset_index(drop=True, level=1)
             .rename('country')           
             ))

In [19]:
df_country_viz_total = df_country[["title", "country"]]
df_country_viz_total = df_country_viz_total.groupby(['country'])["title"].count().reset_index().sort_values('title', ascending= False).head(10)
df_country_viz_total = df_country_viz_total.rename(columns = {"title": "movies_count",})

 
fig1 = px.bar(df_country_viz_total, x='country', y='movies_count', color_discrete_sequence=px.colors.sequential.RdBu,
       title='Top 10 countries with Netflix Content ')


df_country_viz = df_country[["title", "country"]]
df_country_viz = df_country_viz.groupby(['country'])["title"].count().reset_index().sort_values('title', ascending= False).head(10)

df_country_viz1 = df_country[["title", "type", "country"]]
df_country_viz1 = df_country_viz1.groupby(['country', 'type'])["title"].count().reset_index().sort_values('title', ascending= False)
df_country_viz1 = df_country_viz1.rename(columns = {"title": "movies_count",})

final1 = df_country_viz.merge(df_country_viz1, how = 'left', left_on = 'country', right_on = 'country')
final1['percentage'] = (final1['movies_count']/final1['title'])*100
final1['percentage'] = final1['percentage'].round(1)
final1['percent_string'] = final1['percentage'].astype(str)+ '%'


fig2 = px.bar(final1, x='country', y='percentage', color = 'type',
       title='Top 10 countries with Movie/TV show split ')

In [20]:
fig = go.Figure()
fig.add_trace(
    
go.Bar(x= df_country_viz_total['country'], y= df_country_viz_total['movies_count'], marker_color = 'Maroon',
           text = df_country_viz_total['movies_count'], textposition='auto'))

fig.update_layout(title_text = "Top 10 countries with Netflix Content"
                  , yaxis=dict(title='Movies/TV Shows Count'))
fig.show()

final_movie = final1.query("type == 'Movie'")
final_show = final1.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Bar(
    x=  final_movie['country'],
    y= final_movie['percentage'],
    showlegend=True,
    text = final_movie['percent_string'], 
    textposition='auto',
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Bar(
    x= final_show['country'],
    y= final_show['percentage'],
    showlegend=True,
    text = final_show['percent_string'], 
    textposition='auto',
    name='TV Show',
    marker_color='Grey' 
))



# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='stack', title_text = 'Top 10 countries with Movie/TV show split '
                  , yaxis=dict(title='% Movies/TV Shows Count'))
fig.show()

In [21]:
df_2 = df.query("type == 'Movie'")
df_2 = df_2[["title", "rating"]]
df_2 = df_2.groupby(['rating'])["title"].count().reset_index().sort_values('title', ascending = False)
df_2 = df_2.rename(columns = {"title": "movies_count"})
px.bar(df_2, x='rating', y='movies_count', color_discrete_sequence=px.colors.sequential.RdBu,
       title='For which category the maximum content(Movies) are uploaded? ')

United States is the top leaader in both movie and TV shows. India followed US in the overall content and it seems that it has the most number of movies with very less percentage of TV shows comapred to UK and Japan.

In [22]:
df_3 = df.query("type == 'TV Show'")
df_3 = df_3[["title", "rating"]]
df_3 = df_3.groupby('rating')["title"].count().reset_index().sort_values('title', ascending = False)
df_3 = df_3.rename(columns = {"title": "movies_count"})
px.bar(df_3, x='rating', y='movies_count', color_discrete_sequence=['grey'],
       title='For which category the maximum content(TV Shows) are uploaded?')

In [23]:
df_5 = df.query("release_year >= 2007")
df_5 = df_5.groupby("release_year")["show_id"].count().reset_index()

fig = px.area(df_5, x='release_year', y='show_id', color_discrete_sequence=px.colors.sequential.RdBu,
      title='Overall content release Trend')
fig.show()

In 2007, Netflix introduced streaming media and video on demand. We see a slow in the beginning but then it picked up in 2014-2015 and there is a rapid increase till 2018.

By 2018, the content on netlix was 13 times of 2007 year's content. But it has declined since 2019 since the beginning of covid. The other factor could be - In 2019, Disney plus was also launched. Films and television series produced by The Walt Disney Studios and Walt Disney Television, such as Marvel movies moved to Disney plus.

In [24]:
#df_4 = df.query("type == 'Movie'")
df_4 = df.query("release_year >= 2007")
df_4 = df_4.groupby(["type","release_year"])["show_id"].count().reset_index()
df_4_movie = df_4.query("type == 'Movie'")
df_4_show = df_4.query("type == 'TV Show'")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=  df_4_movie['release_year'],
    y= df_4_movie['show_id'],
    showlegend=True,
    text = df_4_movie['show_id'], 
    
    name='Movie',
    marker_color='Maroon'    
    
))
fig.add_trace(go.Scatter(
    x=  df_4_show['release_year'],
    y= df_4_show['show_id'],
    showlegend=True,
    text = df_4_show['show_id'], 
 
    name='TV Show',
    marker_color='Grey' 
))

fig.update_traces( mode='lines+markers')
fig.update_layout(title_text = 'Movies/TV Show release yearly Trend' )
fig.show()

It looks like for both TV shows and movies - the three most common genres are international movies and dramas.

Conclusion

In [26]:
df_genre_viz = df_genre[["title", "type", "listed_in"]]
df_genre_viz = df_genre_viz.groupby(['listed_in', 'type'])["title"].count().reset_index().sort_values('title')
df_genre_viz = df_genre_viz.rename(columns = {"title": "movies_count", "listed_in": "Genre"})

df_genre_movie = df_genre_viz.query("type == 'Movie'")
df_genre_tvshow = df_genre_viz.query("type == 'TV Show'")
# fig1 = px.bar(df_genre_movie, x='movies_count', y='Genre', color_discrete_sequence=px.colors.sequential.RdBu,
#        title='For which Genre the maximum content(Movies) are uploaded? ', height=600)
# fig2 = px.bar(df_genre_tvshow, x='Genre', y='movies_count', color_discrete_sequence=['Grey'],
#        title='For which Genre the maximum content(Shows) are uploaded? ')
#fig1.show() 
#fig2.show()
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'bar'}]],
                   subplot_titles = ['For which Genre the maximum Movies are uploaded?', 'For which Genre the maximum Shows are uploaded? ' ],
                   horizontal_spacing = 0.3)
fig.add_trace(
    
    go.Bar(x= df_genre_movie['movies_count'], y= df_genre_movie['Genre'], orientation = 'h', marker_color='Maroon',  name="Movie",
           text = df_type['count_showids'], textposition='auto'),
    row=1, col=1)

fig.add_trace(
    
    go.Bar(x= df_genre_tvshow['movies_count'], y= df_genre_tvshow['Genre'], orientation = 'h', marker_color = 'Grey', name = "TV Show"),
    row=1, col=2)

fig.update_layout( height = 600)
fig.show()

# Conclusion

We did exploratory data analysis on Netflix Movie Data. We found a lot of insights from the data. This is the first step in our series.

Next, we will engineer useful features and begin developing our recommendation model.

For modelling step,we will start with this data or with the rating data from https://www.kaggle.com/netflix-inc/netflix-prize-data and we will see if it helps for our recommendation system